In [2]:
from scipy.integrate import *
import matplotlib.pyplot as plt
from math import *
import bohrium as np
from scipy.optimize import *

In [3]:
cai = 100.0e-9
cae = 1.0e-6


# Volume Normalisations
vol_er = (3.9*0.1*0.1)
vol_cyt = (4.0*0.5*0.5-vol_er)
vol_tot = vol_cyt + vol_er
ntot = 1
ctot = ntot/vol_tot
fvol_cyt = vol_cyt/vol_tot
fvol_er = vol_er/vol_tot

In [4]:
def f(m):
    return m**(2.0/3.0) #multiplier 

# Reaction Rates
k_orig={
    'kx1_x1a' : 2*1.0e8,
    'kx1a_x2' : 1.0e8,
    'kx1a_x1' : 83.666,
    'kx2_x1a' : 2*83.666,
    'kx2_y2' : 0.6,
    'ky2_x2' : 4.118, #corrected
    'ky2_y1a' : 2*30.015,
    'ky1a_y1' : 30.015,
    'ky1a_y2' : 1.0e5,
    'ky1_y1a' : 2*1.0e5,
    'ky1_x1' : 0.4,
    'kx1_y1' : 1.20e-3,
    'kca' : 0
}
k=k_orig.copy()
execute_string=[]
for key in k.keys():
    execute_string.append(key + " = "+str(k[key]))
exec('; '.join(execute_string))

In [5]:
# SERCA ODE model
def serca_ode(v ,t):
    global k

    fx1, fx1a, fx2, fy2, fy1a, fy1, cae = v

    execute_string=[]
    for key in k.keys():
        execute_string.append("global "+key+"; "+key + " = "+str(k[key]))
    exec('; '.join(execute_string))

    
    dx1 = fx1*(-kx1_x1a*cai-kx1_y1)+fx1a*kx1a_x1+fy1*ky1_x1
    dx1a = fx1a*(-kx1a_x2*cai-kx1a_x1)+fx1*cai*kx1_x1a+fx2*kx2_x1a
    dx2 = fx2*(-kx2_y2-kx2_x1a)+fx1a*cai*kx1a_x2+fy2*ky2_x2

    dy1 = fy1*(-ky1_y1a*cae-ky1_x1)+fy1a*ky1a_y1+fx1*kx1_y1
    dy1a = fy1a*(-ky1a_y2*cae-ky1a_y1)+fy1*cae*ky1_y1a+fy2*ky2_y1a
    dy2 = fy2*(-ky2_x2-ky2_y1a)+fy1a*cae*ky1a_y2+fx2*kx2_y2

    dcae = -cae*(fy1a*ky1a_y2 + fy1*ky1_y1a) + (fy1a*ky1a_y1 + fy2*ky2_y1a) + kca*(cai - cae)
    #print dcae

    return [dx1, dx1a, dx2, dy2, dy1a, dy1, dcae]

In [6]:
def get_SERCA_equilibrium(final_conc):
    global k
    #params for general final conc
    sf=final_conc/250.0
    sm=1
    
    for key in ['kx1_x1a','kx1a_x2']:
        k[key]=k_orig[key]*sf

    cae=final_conc*1e-6
    #arbitrary inital serca
    v0=[9.06424261e-01, 8.66707395e-02, 2.07182701e-03, 3.01868945e-04,
           1.81203090e-03, 2.71927273e-03, cae]
    
    #ODE details
    tstep = 1e-2
    tf = 100
    t = np.linspace(0, tf, tf/tstep+1)

    # Solve ODE
    sol = odeint(serca_ode, v0, t)

    return list(np.array(sol[-1,:-1]))

In [120]:
a="get_SERCA_equilibrium(500)"

[0.6424441123011492,
 0.3071470429089671,
 0.03671109446072891,
 0.005348872466555567,
 0.006421544779120445,
 0.001927333169594788]

In [121]:
a="""def func():
    global k
    execute_string=[]
    for key in k.keys():
        execute_string.append(key + " = "+str(k[key]))
    print ('; '.join(execute_string))
    exec('; '.join(execute_string))
    print (kx1_x1a,kx1a_x2)"""

In [7]:
get_SERCA_equilibrium(100)

[0.9064242610635374,
 0.08667073947013512,
 0.0020718270138444204,
 0.0003018689189853863,
 0.0018120308340165466,
 0.002719272784396905]